In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../") # go to parent dir
from src.data.diversity_document import DiversityDocument
import spacy
nlp = spacy.load('en')
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim

df = pd.read_csv('../data/external/2017-18stats.csv')
with open('../data/external/download_whitelist.txt', 'r') as f:
    whitelist = [line.strip() for line in f]
df['CompanyLinkToGPGInfo'] = df['CompanyLinkToGPGInfo'].astype(str)

number_of_companies = len(df.index)
texts = list()
metadata = list()

In [2]:
#split the corpus into quartiles by the "best" companies on diversity numbers. 
df.head(1)
print(df.DiffMeanHourlyPercent.count())
df.describe()


10159


,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,MaleLowerQuartile,FemaleLowerQuartile,MaleLowerMiddleQuartile,FemaleLowerMiddleQuartile,MaleUpperMiddleQuartile,FemaleUpperMiddleQuartile,MaleTopQuartile,FemaleTopQuartile
count,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000,10159.000000
mean,14.470046,11.963943,15.259415,-13.792430,35.844000,34.328861,46.427434,53.572566,50.636047,49.363953,54.995196,45.004804,60.959228,39.040772
std,15.087346,15.687637,242.144758,1258.967565,36.398861,36.102908,24.114317,24.114317,26.063711,26.063711,26.172304,26.172304,24.316170,24.316170
min,-400.000000,-164.000000,-21092.000000,-120695.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.300000,0.900000,0.000000,0.000000,1.000000,0.400000,27.000000,35.550000,29.600000,27.500000,34.000000,21.200000,41.800000,17.050000
50%,13.500000,9.600000,22.500000,5.700000,20.500000,18.000000,44.400000,55.600000,48.500000,51.500000,54.000000,46.000000,63.000000,37.000000
75%,23.000000,21.100000,53.000000,38.300000,73.800000,70.900000,64.450000,73.000000,72.500000,70.400000,78.800000,66.000000,82.950000,58.200000
max,159.000000,75.700000,3848.200000,5566.700000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


In [6]:
upper_q = df.DiffMeanHourlyPercent.quantile(0.75)
lower_q = df.DiffMeanHourlyPercent.quantile(0.25)
print(upper_q)
print(lower_q)
df_bad = df.loc[df.DiffMeanHourlyPercent > upper_q]
df_good = df.loc[df.DiffMeanHourlyPercent < lower_q]
print(df_bad.DiffMeanHourlyPercent.count())
print(df_good.DiffMeanHourlyPercent.count())

23.0
5.3
2513
2537


In [7]:
for index, row in df_bad.iterrows():
    percent_complete = ((index + 1) / number_of_companies) * 100
    print('%.2f percent complete' % (percent_complete))

    doc = DiversityDocument(row)

    if doc.url_hash in whitelist:
        continue

    if doc.raw_text is not None:
        texts.append(doc.raw_text)
        metadata.append(row.to_dict())

    if not doc.has_downloaded_file and not doc.company_link == 'nan':
        doc.download()
        if doc.raw_text is None:
            with open('../data/external/download_whitelist.txt', 'a') as whitelist_file:
                whitelist_file.write(doc.url_hash + '\n')

0.24 percent complete
0.34 percent complete
0.36 percent complete
0.44 percent complete
0.46 percent complete
0.48 percent complete
0.52 percent complete
0.56 percent complete
0.59 percent complete
0.67 percent complete
0.68 percent complete
0.76 percent complete
0.77 percent complete
0.79 percent complete
0.83 percent complete
0.96 percent complete
0.97 percent complete
0.98 percent complete
1.00 percent complete
1.15 percent complete
1.27 percent complete
1.28 percent complete
1.30 percent complete
1.34 percent complete
1.42 percent complete
1.55 percent complete
1.58 percent complete
1.60 percent complete
1.77 percent complete
1.86 percent complete
1.92 percent complete
2.00 percent complete
2.01 percent complete
2.02 percent complete
2.05 percent complete
2.07 percent complete
2.08 percent complete
2.10 percent complete
2.12 percent complete
2.14 percent complete
2.18 percent complete
2.20 percent complete
2.20 percent complete
2.21 percent complete
2.28 percent complete
2.29 perce

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [8]:
texts[0]

'                Home (current)     About Us       Supply Chain     Technical Expertise     Memberships        Sectors       Commercial Marine     Oil and Gas     Energy     Nuclear     Defence     Fabrication     Marine Structures     Port Operations        Facilities       A&P Tyne     A&P Falmouth     A&P Tees     A&P Great Yarmouth        Global Reach     Work Safe Home Safe     People       Policies     Careers        News     Contact       International Agents     Media Enquiries              Engineering Strength    We are A&P                           Global ship repair, conversion and marine specialist A&P Group has a proud history of quality-driven engineering and marine excellence spanning six decades.\xa0With far-reaching capabilities from our four UK locations, we combine a rich heritage of marine engineering skills and experience to provide ship owners, governments and energy companies with all the precision skills needed to complete the most demanding projects.  As the la

In [ ]:
processed_docs = []
for doc in nlp.pipe(texts, n_threads=2, batch_size=8):
    # Process document using Spacy NLP pipeline.

    ents = doc.ents  # Named entities.

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove stopwords, and single character words
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop and len(token) > 1]

    # Add named entities, but only if they are a compound of more than word.
    doc.extend([str(entity) for entity in ents if len(entity) > 1])

    #processed_docs.append(doc)

#docs = processed_docs
#del processed_docs


In [ ]:
# Add bigrams and trigrams to docs (only ones that appear 5 times or more).
bigram = Phrases(docs, min_count=5)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)


# Create a dictionary representation of the documents, and filter out frequent and rare words.
dictionary = Dictionary(docs)

# Remove rare and common tokens.
# Filter out words that occur too frequently or too rarely.
max_freq = 0.5
min_wordcount = 5
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
dictionary.compactify()

_ = dictionary[0]  # This sort of "initializes" dictionary.id2token.


In [ ]:

# Vectorize data.
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

In [ ]:
lda_model = LdaModel(corpus=corpus,
               id2word=dictionary,
               num_topics=8, 
               random_state=100,
               update_every=1,
               chunksize=100,
               passes=10,
               alpha='auto',
               per_word_topics=True)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 
# a measure of how good the model is. lower the better. -7.670575002821739

from gensim.models.coherencemodel import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)